# TypeScript コーディング問題: Plus One

## 1. 問題の分析

### 競技プログラミング視点での分析
- **実行速度最優先**: 配列を右から左へ1回走査（O(n)）、繰り上がり処理のみ実施
- **メモリ使用量最小化**: 基本的にin-placeで処理、繰り上がりが先頭まで伝播する場合（all 9s）のみ新配列生成
- **最悪ケース**: `[9,9,9]` → `[1,0,0,0]` のケースでO(n)の新規メモリ確保が必要

### 業務開発視点での分析
- **型安全性**: 入力配列の各要素が0-9の範囲内であることを型で保証
- **エラーハンドリング**: 不正な入力（負の値、10以上の値）に対する検証
- **可読性**: 繰り上がりロジックを明確に表現
- **副作用の明示**: 元の配列を変更する（LeetCodeでは許容されるが、業務では注意が必要）

### TypeScript特有の考慮点
- **型推論**: `number[]` の厳密な型定義で0-9の範囲を保証
- **readonly**: 引数を `readonly number[]` にすると副作用を防げる（要新配列生成）
- **型ガード**: 実行時の配列要素検証
- **コンパイル時最適化**: strict modeでのnull安全性確保

## 2. アルゴリズムアプローチ比較

| アプローチ | 時間計算量 | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考 |
|---------|----------|---------|------------|---------|--------|------|
| 右から走査(in-place) | O(n) | O(1)* | 低 | 高 | 高 | *最悪時O(n)、元配列変更 |
| 右から走査(immutable) | O(n) | O(n) | 中 | 高 | 高 | 常に新配列生成 |
| BigInt変換 | O(n) | O(n) | 低 | 中 | 中 | 桁数制限に注意 |
| 再帰的処理 | O(n) | O(n) | 高 | 中 | 低 | スタックオーバーフロー懸念 |

**注**: LeetCode形式では通常in-place変更が許容されるため、最悪時のみO(n)の空間計算量

## 3. 選択したアルゴリズムと理由

### 選択したアプローチ
**右から走査(in-place)方式**

### 理由
- **計算量的な優位性**: 
  - 時間計算量O(n)で最適
  - 空間計算量はほとんどのケースでO(1)、最悪時（all 9s）のみO(n)
- **TypeScript環境での型安全性**:
  - 配列操作が直感的で型推論が効く
  - 繰り上がりフラグを明示的に管理できる
- **保守性・可読性の観点**:
  - 小学校の筆算アルゴリズムと同じロジックで理解しやすい
  - エッジケースが明確（繰り上がり継続 vs 終了）

### TypeScript特有の最適化ポイント
- **早期リターン**: 繰り上がりが発生しない時点で即座に返却
- **型安全な配列操作**: `unshift` より `[1, ...digits]` で可読性向上
- **const assertion**: 定数値の型を厳密化

## 4. 実装コード
Analyze Complexity
Runtime 0 ms
Beats 100.00%
Memory 56.17 MB
Beats 6.38%
```typescript
/**
 * 大きな整数を表す配列に1を加算する
 * @param digits - 各桁を表す数値配列（最上位桁が先頭）
 * @returns 1を加算した結果の配列
 * @complexity Time: O(n), Space: O(1) 平均、O(n) 最悪時
 * @sideEffect 入力配列を直接変更する（LeetCode形式では許容）
 */
function plusOne(digits: number[]): number[] {
    // 型ガード: 配列の検証
    if (!Array.isArray(digits) || digits.length === 0) {
        throw new TypeError('Input must be a non-empty array');
    }
    
    // 右端から左へ走査
    for (let i = digits.length - 1; i >= 0; i--) {
        // 現在の桁が9未満の場合
        if (digits[i] < 9) {
            digits[i]++; // 副作用: 元配列を変更
            return digits; // 繰り上がり不要、即座に返却
        }
        
        // 現在の桁が9の場合、0にして繰り上がり継続
        digits[i] = 0; // 副作用: 元配列を変更
    }
    
    // 全桁が9だった場合（例: [9,9,9] → [1,0,0,0]）
    // この時点で元配列は [0,0,0] に変更済み
    // 先頭に1を追加した新配列を返却
    return [1, ...digits];
}
```

### コード解説

1. **型ガード（オプショナル）**: LeetCodeでは不要だが、業務コードでは有用
2. **右から左へのループ**: 
   - `digits[i] < 9` の場合: インクリメントして即座にリターン（O(1)で終了）
   - `digits[i] === 9` の場合: 0に設定して次の桁へ繰り上がり継続
3. **全桁9のケース**: ループを抜けた = 全桁が9だった → `[1, 0, 0, ..., 0]` を返却

### 副作用に関する重要な注意

**この関数は入力配列を直接変更します（impure function）**：

```typescript
const original = [1, 2, 9];
const result = plusOne(original);
// original は [1, 3, 0] に変更されている（副作用）
// result も [1, 3, 0] を参照（同じ配列）

const allNines = [9, 9, 9];
const result2 = plusOne(allNines);
// allNines は [0, 0, 0] に変更されている（副作用）
// result2 は [1, 0, 0, 0]（新しい配列）
```

**LeetCodeでは許容されますが、業務コードでは以下の対応を検討**：
- 引数を `readonly number[]` にして、内部で `[...digits]` をコピー
- 関数名を `plusOneInPlace` に変更して副作用を明示
- JSDocに `@sideEffect` タグを追加

### 型安全性の特徴

- **入力型**: `number[]` - 数値配列であることを保証
- **戻り値型**: `number[]` - 同じ型を返却
- **副作用**: 元配列を変更する（LeetCode形式では許容）
- **null安全性**: TypeScript strict modeで配列操作が安全

### エッジケース処理

```typescript
// テストケース例（実装には含めない）
// [1,2,3] → [1,2,4]  早期リターン、元配列変更
// [1,9,9] → [2,0,0]  2回繰り上がり、元配列変更
// [9,9,9] → [1,0,0,0] 全桁繰り上がり、元配列は[0,0,0]に変更済み
// [0] → [1]          単一要素、元配列変更
```

## TypeScript固有の最適化観点

### 型安全性の活用
- **厳密な型定義**: `number[]` で配列要素の型を保証
- **配列メソッドの型推論**: スプレッド構文 `[1, ...digits]` での型安全な配列生成
- **境界値チェック**: インデックスアクセスの安全性

### コンパイル時最適化
- **const宣言**: ループカウンタ `i` を `let` で宣言（再代入が必要）
- **早期リターン**: 不要な処理をスキップしてパフォーマンス向上
- **スプレッド構文**: `[1, ...digits]` はコンパイラによって最適化される

### 開発効率と保守性
- **シンプルなロジック**: 筆算のアルゴリズムそのもので直感的
- **副作用の明示**: 関数が元配列を変更することをドキュメント化
- **拡張性**: 他の進数への対応も容易（10を変数化すれば対応可能）

### Immutable版の実装例（業務コード向け）

```typescript
/**
 * 大きな整数を表す配列に1を加算する（Immutable版）
 * @param digits - 各桁を表す数値配列（最上位桁が先頭）
 * @returns 1を加算した結果の新しい配列
 * @complexity Time: O(n), Space: O(n)
 * @pure 元配列を変更しない
 */
function plusOneImmutable(digits: readonly number[]): number[] {
    const result = [...digits]; // コピーを作成
    
    for (let i = result.length - 1; i >= 0; i--) {
        if (result[i] < 9) {
            result[i]++;
            return result;
        }
        result[i] = 0;
    }
    
    return [1, ...result];
}
```